# Representações Discretas de Texto (BoW, N-gram, TF-IDF)

## 1\. Introdução: Representando a Linguagem

No Processamento de Linguagem Natural (PLN), um dos principais desafios é como converter o significado simbólico da linguagem humana (texto) em uma forma que possa ser compreendida por algoritmos de Machine Learning (ML). Essa conversão é chamada de **vetorização**.

O princípio central para isso é o **Modelo de Espaço Vetorial (Vector Space Model - VSM)**, que define a vetorização como o processo de representar documentos como vetores de features (características).

No contexto dos modelos clássicos, a representação de texto envolve a criação de vetores discretos (Sparse).

Dado um vocabulário total $V$ de um corpus (a coleção inteira de documentos), o vetor $v_d$ de um documento $d$ terá a dimensão $|V|$. Como um documento individual contém apenas uma pequena fração das palavras do vocabulário total, a maioria dos valores no vetor $v_d$ será zero, resultando em **vetores esparsos**.

## 2\. Bag-of-Words (BoW): O Saco de Palavras

O Bag-of-Words (BoW), ou Saco de Palavras, é a técnica mais direta e fundamental de representação vetorial em PLN.

### 2.1 Conceito e Mecanismo

  * **Definição:** O BoW trata o texto como uma "bolsa" ou coleção de palavras, **ignorando completamente a ordem das palavras** e a estrutura gramatical ou sintática. É um multiconjunto (multiset) onde cada elemento (palavra) tem uma contagem.
  * **Vetorização:** Cada palavra única no conjunto de dados (o vocabulário) corresponde a uma dimensão (feature index) no vetor. O vetor do documento armazena a frequência de ocorrência de cada palavra nesse documento.

**Exemplos de Representação BoW (Variações):**

| Variação | Valor do Vetor | Fonte |
| :--- | :--- | :--- |
| **BoW de Contagem** | A contagem (frequência) da palavra no documento. | Mais popular |
| **BoW Booleano** | 1 se a palavra estiver presente, 0 caso contrário. | Menos comum |

### 2.2 Implementação Conceitual (Python Puro)

Podemos simular a ideia do BoW usando um simples contador.

In [1]:
# Demonstração Conceitual de BoW (Contagem)
from collections import Counter

texto = "o gato na cartola e a cartola do gato"
tokens = texto.split() # Tokenização simplificada
counts = Counter(tokens)

print("Contagem de Palavras (BoW):\n", counts)

# Vocabulário (mapeamento de palavra para índice):
# {'o': 0, 'gato': 1, 'na': 2, 'cartola': 3, 'e': 4, 'a': 5, 'do': 6}
# Vetor (simulado): [1, 2, 1, 2, 1, 1, 1]

Contagem de Palavras (BoW):
 Counter({'gato': 2, 'cartola': 2, 'o': 1, 'na': 1, 'e': 1, 'a': 1, 'do': 1})


### 2.3 Implementação Prática com `scikit-learn`

Em um pipeline real, usamos `CountVectorizer` do `scikit-learn` para criar a matriz Documento-Termo (Document-Term Matrix).

In [2]:
# Implementação Prática com CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Nosso "corpus" (coleção de documentos)
corpus = [
    'o gato na cartola',
    'a cartola do gato',
    'o gato e o cachorro'
]

# 1. Inicializar o vetorizador
# (vamos incluir stop_words em português para uma limpeza básica)
vectorizer_bow = CountVectorizer(stop_words=['o', 'na', 'e', 'a', 'do', 'da'])

# 2. Aprender o vocabulário e transformar o corpus
X_bow = vectorizer_bow.fit_transform(corpus)

# 3. Visualizar o resultado
print("Vocabulário (Features):")
print(vectorizer_bow.get_feature_names_out())
print("\nMatriz Esparsa (Documento, Termo):")
print(X_bow.toarray())

# Visualização como DataFrame para clareza
pd.DataFrame(X_bow.toarray(), columns=vectorizer_bow.get_feature_names_out(), index=['Doc 1', 'Doc 2', 'Doc 3'])

Vocabulário (Features):
['cachorro' 'cartola' 'gato']

Matriz Esparsa (Documento, Termo):
[[0 1 1]
 [0 1 1]
 [1 0 1]]


,cachorro,cartola,gato
Doc 1,0,1,1
Doc 2,0,1,1
Doc 3,1,0,1


### 2.4 Desvantagens e Limitações do BoW

O BoW, embora simples, apresenta limitações críticas:

1.  **Visão Plana do Documento (Flattened View):** O BoW perde completamente a ordem e a estrutura sintática.
      * `"o cavalo comeu"` e `"comeu o cavalo"` teriam o **mesmo vetor** BoW.
2.  **Maldição da Dimensionalidade (Curse of Dimensionality):** Se o vocabulário for muito grande (e.g., \> 100.000 palavras), o vetor terá milhares de dimensões, sendo a maioria zeros (esparsidade).
3.  **Similaridade Semântica (Ortogonalidade):** Não há conceito de similaridade de significado. No espaço BoW, a distância entre palavras sinônimas é a mesma que entre palavras não relacionadas.
      * $d(\text{cão, gato}) = d(\text{cão, cadeira})$

## 3\. N-grams: Adicionando Contexto Local

A abordagem BoW falha em incorporar a ordem das palavras. Os N-grams (ou Bag-of-N-Grams - BoN) são usados para resolver parcialmente esta questão, capturando algum contexto local.

### 3.1 Conceito e Tipos

  * **Definição:** Um N-gram é uma subsequência de $N$ palavras contíguas.
  * **Mecanismo:** O vocabulário $V$ é estendido para incluir todos os $N$-grams únicos. O vetor do documento armazena as contagens desses $N$-grams.
  * **Tipos Comuns:**
      * **Unigramas (N=1):** Equivale ao BoW. (`"gato"`, `"cartola"`)
      * **Bigramas (N=2):** Combinações de duas palavras. (`"gato na"`, `"na cartola"`)
      * **Trigramas (N=3):** Combinações de três palavras. (`"gato na cartola"`)

### 3.2 Dimensionalidade e Escolha de N

  * **Aumento da Esparsidade:** Usar $N$ maior (e.g., trigramas) captura mais contexto, mas aumenta *exponencialmente* o tamanho do vocabulário e a esparsidade do vetor.
  * **Escolha de N:** Geralmente $N < 4$. Para textos curtos (tweets), bigramas são comuns. Para documentos mais longos, uma combinação de unigramas e bigramas (`ngram_range=(1, 2)`) é um ótimo ponto de partida.

### 3.3 Implementação Prática com `scikit-learn`

Não precisamos de um loop manual. O `CountVectorizer` lida com N-grams através do parâmetro `ngram_range`.

In [7]:
# Implementação Prática de N-grams (Bigramas)

# Vamos usar o mesmo corpus, mas agora pedindo unigramas E bigramas
# ngram_range=(1, 2) significa "incluir N=1 e N=2"
vectorizer_ngram = CountVectorizer(stop_words=['o', 'na', 'e', 'a', 'do', 'da'], ngram_range=(1, 2))

# Aprender o vocabulário e transformar
X_ngram = vectorizer_ngram.fit_transform(corpus)

# Visualizar o resultado
print("Vocabulário (Unigramas + Bigramas):")
print(vectorizer_ngram.get_feature_names_out())

print("\nMatriz Esparsa (Documento, N-gram):")
pd.DataFrame(X_ngram.toarray(), columns=vectorizer_ngram.get_feature_names_out(), index=['Doc 1', 'Doc 2', 'Doc 3'])

Vocabulário (Unigramas + Bigramas):
['cachorro' 'cartola' 'cartola gato' 'gato' 'gato cachorro' 'gato cartola']

Matriz Esparsa (Documento, N-gram):


,cachorro,cartola,cartola gato,gato,gato cachorro,gato cartola
Doc 1,0,1,0,1,0,1
Doc 2,0,1,1,1,0,0
Doc 3,1,0,0,1,1,0


## 4\. TF-IDF: Ponderando a Relevância do Termo

O TF-IDF (Term Frequency-Inverse Document Frequency) resolve uma deficiência central do BoW de contagem: palavras muito comuns (como "o", "de", "que") dominam a contagem, mesmo sem carregar significado discriminativo.

O TF-IDF quantifica a **importância** de uma palavra combinando dois fatores:

1.  **Frequência do Termo (Term Frequency - TF):** Quão frequente é a palavra *neste documento*?
2.  **Frequência Inversa do Documento (Inverse Document Frequency - IDF):** Quão *rara* é a palavra *em todo o corpus*?

### 4.1 Formulação Matemática

O TF-IDF é o produto dos dois.

#### 4.1.1 Term Frequency (TF)

O TF($w_i, d$) é o número de vezes que a palavra $w_i$ aparece no documento $d$. (Frequentemente normalizado ou suavizado logaritmicamente).

$$
\text{TF}(w_i, d) = f_{w_i, d}
$$#### 4.1.2 Inverse Document Frequency (IDF)

O IDF($w_i$) atribui um peso maior a termos que são raros no corpus. Palavras que aparecem em *todos* os documentos (como stopwords) terão um IDF próximo de zero.

$$\text{IDF}(w\_i) = \log \left( \frac{N}{\text{df}(w_i)} \right)
$$Onde $N$ é o número total de documentos no corpus e $\text{df}(w_i)$ (Document Frequency) é o número de documentos que contêm a palavra $w_i$.

*(Nota: O `scikit-learn` usa uma versão suavizada "smooth" por padrão, adicionando +1 ao numerador e denominador para evitar divisão por zero).*

#### 4.1.3 TF-IDF Score

O score final equilibra a frequência local (TF) com a raridade global (IDF).

$$
\text{TFIDF}(w_i, d) = \text{TF}(w_i, d) \times \text{IDF}(w_i)
$$Um valor **alto** de TF-IDF indica que o termo é frequente *naquele documento*, mas raro *no corpus total*, tornando-o um bom discriminador.

### 4.2 Implementação Prática com `scikit-learn`

Usamos o `TfidfVectorizer`, que combina o `CountVectorizer` com a transformação TF-IDF.

In [24]:
# Implementação Prática com TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Nosso "corpus" (coleção de documentos)
#corpus = [
#    'o gato na cartola',
#    'a cartola do gato',
#    'o gato e o cachorro'
#]

# Nosso "corpus" (coleção de documentos)
corpus = [
    'o gato na cartola',
    'a cartola do gato',
    'o gato na PUC. Churrasco de gato do bandeco na PUC é bom!',
    'o gato e o cachorro'
]

# Usando o mesmo corpus
# (O TfidfVectorizer também pode calcular N-grams e remover stop_words)
#tfidf_vectorizer = TfidfVectorizer(stop_words=['o', 'na', 'e', 'a', 'do', 'da', 'de'])
tfidf_vectorizer = TfidfVectorizer(stop_words=['o', 'na', 'e', 'a', 'do', 'da', 'de'])

# Aprender vocabulário, contar e aplicar a transformação TF-IDF
X_tfidf = tfidf_vectorizer.fit_transform(corpus)

# Visualizar o resultado
print("Vocabulário (Features):")
print(tfidf_vectorizer.get_feature_names_out())

print("\nMatriz TF-IDF (Documento, Feature):")
# Note que os valores agora são pesos (floats), não contagens (inteiros)
pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=['Doc 1', 'Doc 2', 'Doc 3', 'Doc 4'])

Vocabulário (Features):
['bandeco' 'bom' 'cachorro' 'cartola' 'churrasco' 'gato' 'puc']

Matriz TF-IDF (Documento, Feature):


,bandeco,bom,cachorro,cartola,churrasco,gato,puc
Doc 1,0.000000,0.000000,0.000000,0.833884,0.000000,0.551939,0.000000
Doc 2,0.000000,0.000000,0.000000,0.833884,0.000000,0.551939,0.000000
Doc 3,0.351597,0.351597,0.000000,0.000000,0.351597,0.366956,0.703194
Doc 4,0.000000,0.000000,0.886548,0.000000,0.000000,0.462637,0.000000


*Observação: Note como "gato" e "cartola", que aparecem em 2 de 3 documentos, têm um peso menor do que termos únicos de um documento, como "cachorro".*

### 4.3 Uso e Limitações do TF-IDF

O TF-IDF é um *baseline* (linha de base) extremamente forte para tarefas de classificação de texto (e.g., análise de sentimento, classificação de notícias) quando usado com algoritmos clássicos (Naive Bayes, SVM, Regressão Logística).

Contudo, o TF-IDF herda as principais fraquezas da representação Bag-of-Words:

* **Insensibilidade ao Contexto:** A ordem das palavras ainda é perdida (a menos que se use N-grams, que captura apenas contexto local).
* **Maldição da Dimensionalidade:** Ainda sofre com vetores longos e esparsos.
* **Ortogonalidade:** Ainda não há noção de similaridade semântica (e.g., "cão" e "cachorro" são features completamente diferentes).

## 5\. Próximos Passos: Indo Além das Representações Discretas

As representações BoW, N-gram e TF-IDF são modelos baseados em contagem que criam vetores **discretos e esparsos**.

As limitações inerentes a esses métodos—especialmente a perda de contexto/ordem e a falta de similaridade semântica—motivaram o desenvolvimento de modelos mais avançados.

O próximo passo é a transição para **Representações Distribuídas (Distributed Representations)**, também conhecidas como **Word Embeddings**. Esses modelos (como Word2vec, GloVe e BERT) usam vetores **densos** (com poucas dimensões, e.g., 300) para capturar o significado semântico e contextual das palavras.
$$